In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


In [36]:
# Read Data
train = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/train data/data.xlsx")
test = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/test data/test.xlsx")

In [38]:
test = test.fillna(0)

In [5]:
# Dummy Lable Dataframe
count = list(range(0,800))
df_label = pd.DataFrame(columns=['主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺尖肋膜增厚','心臟肥大'],index=count)
df_label = df_label.fillna(0)
# Dummy Lable
# '主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺間肋膜增厚','心臟肥大'
for i in range(800):
    #print(i,train['label'][i])
    row_label = train['label'][i]
    if row_label == 'normal':
        continue
    if '主動脈硬化(鈣化)' in row_label:
        df_label['主動脈硬化(鈣化)'][i] = 1
    if '肺尖肋膜增厚' in row_label:
        df_label['肺尖肋膜增厚'][i] = 1
    if '動脈彎曲' in row_label:
        df_label['動脈彎曲'][i] = 1
    if '肺野異常' in row_label:
        df_label['肺野異常'][i] = 1
    if '肺紋增加' in row_label:
        df_label['肺紋增加'][i] = 1
    if '脊椎病變' in row_label:
        df_label['脊椎病變'][i] = 1
    if '心臟肥大' in row_label:
        df_label['心臟肥大'][i] = 1

In [6]:
df_label

,主動脈硬化(鈣化),動脈彎曲,肺野異常,肺紋增加,脊椎病變,肺尖肋膜增厚,心臟肥大
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
795,0,1,0,0,1,0,0
796,0,1,0,0,1,0,0
797,0,1,0,0,1,0,0
798,0,1,0,0,1,0,0


In [ ]:
# dicom to jpg
import cv2 as cv
import numpy as np
import pydicom as PDCM

def Dicom_to_Image(Path):
    DCM_Img = PDCM.read_file(Path)

    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = int(DCM_Img.get(0x00281050).value) #Get window center from tag (0028, 1050)
    Window_Width = int(DCM_Img.get(0x00281051).value) #Get window width from tag (0028, 1051)

    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    return New_Img, Instance_Number

def main():
    # Input Path
    input_path = '/content/drive/MyDrive/深度學習於醫學影像/train data/'
    for i in range(800):
      print(i,"start")
      Input_Image = input_path + train['category'][i] + '/' + train['Filename'][i]
      #Input_Image = '/content/drive/MyDrive/深度學習於醫學影像/train data/normal/220.dcm'

      Output_Image, Instance_Number = Dicom_to_Image(Input_Image)

      cv.imwrite('/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'+train['Filename'][i]+'.jpg', Output_Image)
      print(i,"finished")

if __name__ == "__main__":
    main()

In [17]:
# Read images and merge them into the training data X.
from tensorflow.keras.applications.vgg16 import preprocess_input
X = np.array([])
path = '/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'
for i in range(800):
  print(i)
  input_path = path + train['Filename'][i] + '.jpg'
  print(input_path)
  # Load image and Resize into (224, 224) 
  img = image.load_img(input_path, target_size=(224, 224))
  # Show image
  #imgplot = plt.imshow(img)
  #plt.show()
  # Add one dimension and Become to (1, 224, 224, 3), which the last dimension is color.
  img2 = image.img_to_array(img)
  img2 = np.expand_dims(img2, axis=0)
  #print(img2.shape)
  if len(X.shape) == 1:
      X = img2
  else:
      # Combine the pixels of all images
      X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)
    
np.save('x_ray', X)

0
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_243 (2).dcm.jpg
1
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_97.dcm.jpg
2
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_39.dcm.jpg
3
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_273 (2).dcm.jpg
4
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_96.dcm.jpg
5
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_95.dcm.jpg
6
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_38.dcm.jpg
7
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_272 (2).dcm.jpg
8
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_363.dcm.jpg
9
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_271 (2).dcm.jpg
10
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_76.dcm.jpg
11
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_94.dcm.jpg
12
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_37.dcm.jpg
13
/content/drive/MyDrive/深度學習於醫學影像/train data/tra

In [18]:
from tensorflow.keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)

In [19]:
X.shape

(800, 224, 224, 3)

In [20]:
y = np.array(df_label)

In [ ]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

In [14]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=3))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 100352)            23587712  
_________________________________________________________________
dense (Dense)                (None, 512)               51380736  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 3591      
Total params: 75,234,695
Trainable params: 51,646,983
Non-trainable params: 23,587,712
___________________________________

In [15]:
model.fit(X, y, epochs=10, validation_split=0.2, verbose=2)

Epoch 1/10
20/20 - 132s - loss: 0.2624 - accuracy: 0.2328 - val_loss: 0.9973 - val_accuracy: 0.0375
Epoch 2/10
20/20 - 128s - loss: 0.1550 - accuracy: 0.2844 - val_loss: 0.8228 - val_accuracy: 0.0688
Epoch 3/10
20/20 - 128s - loss: 0.1299 - accuracy: 0.2562 - val_loss: 0.8926 - val_accuracy: 0.0188
Epoch 4/10
20/20 - 127s - loss: 0.1074 - accuracy: 0.2922 - val_loss: 0.9779 - val_accuracy: 0.0188
Epoch 5/10
20/20 - 128s - loss: 0.1089 - accuracy: 0.2266 - val_loss: 0.9630 - val_accuracy: 0.0375
Epoch 6/10
20/20 - 128s - loss: 0.0963 - accuracy: 0.2250 - val_loss: 0.9475 - val_accuracy: 0.0312
Epoch 7/10
20/20 - 128s - loss: 0.0905 - accuracy: 0.2719 - val_loss: 1.1272 - val_accuracy: 0.0500
Epoch 8/10
20/20 - 129s - loss: 0.0897 - accuracy: 0.2844 - val_loss: 1.0723 - val_accuracy: 0.0375
Epoch 9/10
20/20 - 129s - loss: 0.0912 - accuracy: 0.2234 - val_loss: 1.0121 - val_accuracy: 0.1000
Epoch 10/10
20/20 - 126s - loss: 0.0886 - accuracy: 0.2500 - val_loss: 1.1526 - val_accuracy: 0.0188

In [39]:
#/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg10_0 (2).dcm.jpg
test_path = '/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg'    
for n in range(100):
  output_path = test_path + test['Filename'][n]+ '.jpg'
  print(output_path)
  img = image.load_img(output_path, target_size=(224, 224))
  #imgplot = plt.imshow(img)
  #plt.show()
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model_matrix = model.predict(x)
  print("n = ",n)
  print("model_matrix = ",model_matrix)
  test['主動脈硬化(鈣化)'][n] = model_matrix[0][0]
  test['動脈彎曲'][n] = model_matrix[0][1]
  test['肺野異常'][n] = model_matrix[0][2]
  test['肺紋增加'][n] = model_matrix[0][3]
  test['脊椎病變'][n] = model_matrix[0][4]
  test['肺間肋膜增厚'][n] = model_matrix[0][5]
  test['心臟肥大'][n] = model_matrix[0][6]

/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_28.dcm.jpg
n =  0
model_matrix =  [[9.3585044e-07 5.2552077e-06 1.4797106e-06 1.8503351e-06 6.2846266e-07
  8.2622471e-05 5.6919375e-06]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_9.dcm.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipyk

n =  1
model_matrix =  [[5.0702691e-04 9.8040473e-05 6.2466475e-05 4.4053173e-05 4.6713071e-06
  2.7409196e-04 1.0624385e-04]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_8.dcm.jpg
n =  2
model_matrix =  [[5.6198201e-05 1.9767880e-04 5.8354348e-05 1.4467647e-05 6.8033328e-06
  4.6199560e-04 6.7068788e-05]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_7.dcm.jpg
n =  3
model_matrix =  [[3.8751929e-07 3.7050686e-06 9.5459336e-07 1.0850450e-07 8.5777501e-08
  1.4660610e-05 1.6281086e-06]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_6.dcm.jpg
n =  4
model_matrix =  [[1.2136745e-05 6.1662959e-06 8.3700061e-06 4.4090139e-06 8.2893331e-07
  1.8289685e-04 6.6110488e-06]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_1.dcm.jpg
n =  5
model_matrix =  [[3.5558365e-05 3.9592520e-05 1.3104081e-04 1.7331648e-05 5.8453352e-06
  1.3666451e-03 8.0983154e-05]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_2.dcm.jpg
n =  6
model_matrix =  [[1.1466742e-03 8.1798

In [40]:
test

,ID,Filename,主動脈硬化(鈣化),動脈彎曲,肺野異常,肺紋增加,脊椎病變,肺間肋膜增厚,心臟肥大
0,TDR0120171108100516,A0_28.dcm,9.358504e-07,5.255208e-06,1.479711e-06,1.850335e-06,6.284627e-07,8.262247e-05,5.691938e-06
1,TDR0120171130152723,A0_9.dcm,5.070269e-04,9.804047e-05,6.246648e-05,4.405317e-05,4.671307e-06,2.740920e-04,1.062439e-04
2,TDR0120171202104940,A0_8.dcm,5.619820e-05,1.976788e-04,5.835435e-05,1.446765e-05,6.803333e-06,4.619956e-04,6.706879e-05
3,TDR0120171202111823,A0_7.dcm,3.875193e-07,3.705069e-06,9.545934e-07,1.085045e-07,8.577750e-08,1.466061e-05,1.628109e-06
4,TDR0120171207092217,A0_6.dcm,1.213674e-05,6.166296e-06,8.370006e-06,4.409014e-06,8.289333e-07,1.828969e-04,6.611049e-06
...,...,...,...,...,...,...,...,...,...
95,TDR0520180226145555,220_173.dcm,1.568349e-06,1.239978e-06,9.430168e-07,1.215458e-07,3.875899e-07,2.921590e-06,2.902218e-06
96,TDR0520180226145713,220_169.dcm,1.952513e-05,1.871586e-04,1.435280e-04,3.058167e-06,1.871645e-06,8.546986e-05,3.693488e-05
97,TDR0520180226154656,220_168.dcm,6.860685e-07,2.020780e-06,6.859167e-07,2.116660e-08,2.301828e-08,1.029435e-06,6.939961e-07
98,TDR0520180227153750,220_163.dcm,1.691978e-07,4.280705e-06,1.308382e-06,3.092632e-08,1.554583e-08,1.160997e-05,4.846399e-07


In [41]:
# Save VGG16 result
test.to_csv("/content/drive/MyDrive/深度學習於醫學影像/ResNet50_new.csv")  